# Introduction

In [ ]:
# Youtube video: https://www.youtube.com/watch?v=doDUihpj6ro&t=604s
# GitHub Link: https://github.com/nicknochnack/ActionDetectionforSignLanguage/blob/main/Action%20Detection%20Refined.ipynb

# Modules Import

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# Mediapipe keypoints detection and augmentation on the frame

In [ ]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [ ]:
def draw_landmarks(image, results):
    # # Draw face connections
    # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) 
    # # Draw pose/shoulder connections
    # mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) 
    # Draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) 

In [ ]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
    #                          mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
    #                          mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
    #                          )
    # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
    #                          mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
    #                          mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
    #                          ) 
    # Draw pose/ shoulder connections
    # mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
    #                          mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
    #                          mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
    #                          ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

# Camera Feed

In [ ]:
# 0 for all PCs, I placed 1 because of having front and well as back camera in PC
from global_variables import CAMERA_NUMBER, WINDOW_NAME, CAMERA_FEED_EXIT_CHAR
feed = cv2.VideoCapture(CAMERA_NUMBER)
# WINDOW_NAME = 'Webcam feed'
# CAMERA_FEED_EXIT_CHAR = 'q'

In [ ]:
# This cell might not run sometimes, theres no bug, it just needs to be run again if the window opens and closes

# Set mediapipe model and start the webcam
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic_model:
    while feed.isOpened():

        # Read image from the feed of the webcam
        return_value, frame = feed.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic_model)
        # print(results)

        # Draw landmarks
        # draw_landmarks(image, results)
        draw_styled_landmarks(image, results)

        image = cv2.flip(image, 1)
        
        cv2.imshow(WINDOW_NAME, image)

        if cv2.waitKey(10) & 0xFF == ord(CAMERA_FEED_EXIT_CHAR):
            break

    feed.release()
    cv2.destroyAllWindows()

In [ ]:
# Ability to disable and destroy all camera feeds and Open CV window instances
feed.release()
cv2.destroyAllWindows()

# Keypoints extraction

In [ ]:
def extract_keypoints(results):
    # if results.pose_landmarks:
    #     pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten()
    # else:
    #     print("No pose was detected")
    #     pose = np.zeros(132)

    # if results.face_landmarks:
    #     face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten()
    # else:
    #     print("No face was detected")
    #     face = np.zeros(1404)

    if results.left_hand_landmarks:
        left_hand = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten()
    else:
        print("Left hand was not detected")
        left_hand = np.zeros(21*3)

    if results.right_hand_landmarks:
        right_hand = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten()
    else:
        print("Right hand was not detected")
        right_hand = np.zeros(21*3)

    # return np.concatenate([pose, face, left_hand, right_hand])
    return np.concatenate([left_hand, right_hand])

In [ ]:
# extract_keypoints(results)
extract_keypoints(results).shape

# Retrieving actions that need to be detected

In [ ]:
# Actions that we try to detect
import gestures_functions as gf
actions = gf.load_gestures()
print(actions)

# Folder setup for Data Collection

In [ ]:
import os
from global_variables import TRAINING_DATA_FOLDER_NAME, NUMBER_OF_VIDEOS_FOR_EACH_GESTURE, EACH_VIDEO_FRAME_LENGTH

# Path for exported data, numpy arrays
DATA_PATH = os.path.join(TRAINING_DATA_FOLDER_NAME)

# Thirty videos worth of data
num_of_videos = NUMBER_OF_VIDEOS_FOR_EACH_GESTURE

# Videos are going to be 30 frames in length
video_length = EACH_VIDEO_FRAME_LENGTH #frames

# Folder start
start_folder = 30

In [ ]:
for action in actions: 
    try:
        dirmax = np.max(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int))
    except:
        dirmax = 0
    for sequence in range(num_of_videos):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(dirmax+sequence)))
        except:
            pass

# Collect Keypoint Values for Training and Testing

In [ ]:
green = (0, 255, 0)
blue = (0, 0, 255)

announcement_position = (120, 200)
announcement_font_size = 1
announcement_font = cv2.FONT_HERSHEY_SIMPLEX
announcement_color = green
annoucement_line_width = 4
annoucement_line_type = cv2.LINE_AA

text_position = (15, 12)
text_font_size = 0.5
text_font = cv2.FONT_HERSHEY_SIMPLEX
text_color = blue
text_line_width = 1
text_line_type = cv2.LINE_AA

#### Main data collection code snippet

In [ ]:
from global_variables import FRAME_COLLECTION_WAIT_TIME, CAMERA_NUMBER

feed = cv2.VideoCapture(CAMERA_NUMBER)

# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # Loop through actions
    for action in actions:
        # Loop through videos
        for sequence in range(num_of_videos):
            # Loop through video length aka sequence length
            for frame_num in range(video_length):

                # Read feed
                ret, frame = feed.read()

                if ret != True:
                    print(f"Something is wrong!. Camera feed not accessible")
                    break

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(
                        image, 
                        'STARTING COLLECTION', 
                        announcement_position, 
                        announcement_font, 
                        announcement_font_size, 
                        announcement_color, 
                        annoucement_line_width, 
                        annoucement_line_type
                    )
                    cv2.putText(
                        image, 
                        f'Collecting frames for "{action}" video Number {sequence}', 
                        text_position, 
                        text_font, 
                        text_font_size, 
                        text_color, 
                        text_line_width, 
                        text_line_type
                    )
                    # Show to screen
                    cv2.imshow(WINDOW_NAME, image)
                    cv2.waitKey(FRAME_COLLECTION_WAIT_TIME)
                else: 
                    cv2.putText(
                        image, 
                        f'Collecting frames for "{action}" video number {sequence}', 
                        text_position, 
                        text_font, 
                        text_font_size, 
                        text_color, 
                        text_line_width, 
                        text_line_type
                    )
                    # Show to screen
                    cv2.imshow(WINDOW_NAME, image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                full_path_to_frame = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(full_path_to_frame, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord(CAMERA_FEED_EXIT_CHAR):
                    break
                    
    feed.release()
    cv2.destroyAllWindows()

# Preprocess data and create labels and features

In [ ]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}
label_map

In [ ]:
videos, labels = [], []
for action in actions:
    for video in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        video_window = []
        for frame_num in range(video_length):
            res = np.load(os.path.join(DATA_PATH, action, str(video), f"{frame_num}.npy"))
            video_window.append(res)
        videos.append(video_window)
        labels.append(label_map[action])

In [ ]:
np.array(videos).shape  #should be (num_of_video * number of actions, video_length, num of features equal to (the shape of results) all key points identified by mediapipe in each frame)
np.array(labels).shape  #should be num_of_video * number of actions

#### Finally getting the X features and their desired Y labels for training

In [ ]:
X = np.array(videos)
y = to_categorical(labels).astype(int)

In [ ]:
X.shape

In [ ]:
labels

In [ ]:
y

#### Splitting data into training and testing

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [ ]:
X_train.shape
X_test.shape

In [ ]:
y_train.shape
y_test.shape

# Build and Train LSTM Neural Network

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import TensorBoard
from global_variables import LOGGER_FOLDER_NAME

In [ ]:
log_dir = os.path.join(LOGGER_FOLDER_NAME)
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(actions.shape[1],actions.shape[2])))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
actions.shape

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
from global_variables import NUM_OF_EPOCHS

In [ ]:
import tensorboard_training_monitor as monitor
monitor.launch_tensorboard_monitor()
model.fit(X_train, y_train, epochs=NUM_OF_EPOCHS, callbacks=[tb_callback])

In [ ]:
model.summary()

# Make predictions

In [ ]:
results = model.predict(X_test)

In [ ]:
the_index_of_X_test_you_wanna_manually_test = 0

In [ ]:
# should be an array of size equal to number of actions
# this array will be hosting values that add up to 1 where the max value means that the model thinks 
# that this is the most possible solution
first_X_test_result = result[the_index_of_X_test_you_wanna_manually_test]
first_y_test_result = y_text[the_index_of_X_test_you_wanna_manually_test]

In [ ]:
position_of_predicted_action = np.argmax(first_X_test_result)
position_of_actual_action = np.argmax(first_y_test_result)

In [ ]:
predicted_action = actions[position_of_predicted_action]

In [ ]:
actual_action = actions[position_of_actual_action]

# Save Model

In [ ]:
model_name = input("Enter the name of the model: ")

In [ ]:
model.save(f'{model_name}.h5')